## Proyecto Music Stream

**Para la profesora:** 👩🏻‍🏫
Explicar cual es el objetivo del proyecto y cuales son las fases por las que pasará el proyecto.

### Objetivo

1. Crear una bases de datos para almacenar información de canciones y artistas. 
2. Extraer información de la base de datos para responder preguntas sobre la música almacenada.

### Fases del proyecto 

![](./assets/esquema_de_fases.png)

**Fase 1: Extraer Datos**
1. Extraer información de Api Spotify
2. Extraer información de API Last FM
3. Guardar esa información en CSV

**Fase 2:Bases de datos**
1. Diseñar la Base de Datos
2. Crear la bases de datos
3. Insertar los valores en la bases de datos
   
**Fase 3:Extraer información**
1. Extraer información de la base de datos para responder preguntas sobre la música almacenada.
   

## Comencemos... Fase 1.1 Extraer información de Api Spotify


In [2]:
## - Explicar como usar la libreria spotipy 
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [29]:
import pandas as pd
import requests

In [4]:
# Coloca tus credenciales aquí
CLIENT_ID = 'a018711755a146b2a678a93741dc6041'
CLIENT_SECRET = '51fa44e4b57343f19c810b9a75089da6'

# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

### Como usar el método de search de Spotipy

In [5]:
datos = sp.search(q="genre:rock year:2020", type='track', limit=50, offset=0)

In [ ]:
datos

In [22]:
genres = ["pop" , "rock","jazz","hip hop"]
year_start = 2002
year_end = 2007

lista_canciones_albumes = [] 
lista_artistas = []

In [7]:
datos['tracks']['items'][0]['album']['type']


'album'

In [51]:
for genre in genres: 
    datos = sp.search(q=f"genre:{genre} year:{year_start}-{year_end}", type='track,album', limit=50, offset=301)
    datos
    for item in datos['tracks']['items']: 
        
        lista_canciones_albumes.append(
            {
                "nombre_cancion":item['name'],
                "nombre_artista":item['artists'][0]['name'],
                'genre':genre,                 
                "type":  item['album']['type'],
                "year": item['album']['release_date'][:4],
                "id": item['id']
            }
        )
        lista_artistas.append(
            {
                "nombre_artista":item['artists'][0]['name'],
            }
        )
   

In [52]:
len(lista_artistas)

1800

In [53]:
len(lista_canciones_albumes)

1800

In [54]:
#guardar en un csv
df_lista = pd.DataFrame(lista_canciones_albumes)

df_lista.to_csv("canciones.csv", index= False)

## guardo ellistao de artistas en un csv. Verificar que los artistas sea unico

In [55]:
df_lista_artistas = pd.DataFrame(lista_artistas)
df_lista_artistas.to_csv("artistas.csv", index = False)